In [1]:
%load_ext lab_black
%cd ../../..

/mnt/h/hev/log-analytics


In [2]:
import argparse
import math
import multiprocessing
import sys
from datetime import datetime
from pathlib import Path
from pprint import pformat

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_optimizer
import yaml
from easydict import EasyDict
from pytorch_transformers import DistilBertForSequenceClassification, DistilBertTokenizer

from datasets.dataset_ver8 import DatasetGeneratorVer8
from utils import SAM, AverageMeter, CustomLogger, FocalLoss, seed_everything

from main_ver8 import MyTrainer
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm

In [3]:
postfix = "distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4"
outdir = Path("results/distilbert-base-uncased-ver8")
fold = 4

In [4]:
with open("config/distilbert-base-uncased-ver8.yaml", "r") as f:
    C = EasyDict(yaml.load(f, yaml.FullLoader))
    C.result_dir = Path(C.result_dir)
    C.dataset.dir = Path(C.dataset.dir)
    seed_everything(C.seed, deterministic=False)

In [5]:
C

{'model': {'name': 'distilbert-base-uncased'},
 'comment': None,
 'result_dir': PosixPath('results/distilbert-base-uncased-ver8'),
 'debug': False,
 'seed': 20210425,
 'ver': 8,
 'train': {'SAM': False,
  'folds': [4],
  'checkpoints': [None],
  'loss': {'name': 'focal',
   'params': {'gamma': 2.0, 's': 45.0, 'm': 0.1, 'crit': 'focal'}},
  'optimizer': {'name': 'AdamW'},
  'finetune': {'do': True, 'step1_epochs': 2, 'step2_epochs': 4},
  'max_epochs': 12,
  'lr': 1e-05,
  'scheduler': {'name': 'ReduceLROnPlateau',
   'params': {'factor': 0.5, 'patience': 3, 'verbose': True}}},
 'dataset': {'dir': PosixPath('data/ver6'),
  'batch_size': 35,
  'num_workers': 8,
  'oversampling': True,
  'oversampling_scale': 10,
  'augmentation': {'EDA_RD': True}}}

In [6]:
trainer = MyTrainer(C, fold, outdir / f"{postfix}.pth")

Load pretrained results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4.pth


In [7]:
model = trainer.model
model.eval()
torch.set_grad_enabled(False)

In [8]:
activation = []


def hook(model, input, output):
    activation.append(output.detach().cpu())

In [9]:
model.pre_classifier.register_forward_hook(hook)

In [10]:
train_dl = trainer.dsgen.train_only()

# 덱만들기 (Train)

In [11]:
activation = []
deck = {"feat": [], "otext": [], "tlevel": [], "fclevel": []}
with tqdm(total=len(train_dl.dataset), ncols=100, file=sys.stdout) as t:
    for text, tlevel, otext in train_dl:
        fclevel = model(text.cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["tlevel"].append(tlevel)
        deck["otext"].extend(otext)
        t.update(text.size(0))

100%|██████████████████████████████████████████████████████| 421079/421079 [21:29<00:00, 326.65it/s]


In [12]:
deck["feat"] = torch.cat(activation)
deck["tlevel"] = torch.cat(deck["tlevel"])
deck["fclevel"] = torch.cat(deck["fclevel"])

In [13]:
deck["tlevel"].shape, deck["fclevel"].shape

(torch.Size([421079]), torch.Size([421079]))

In [14]:
deck["feat"].shape

torch.Size([421079, 768])

In [15]:
torch.save(deck, outdir / f"{postfix}-deck1.pth")

# Valid Lv7에 대한 결과 분석

In [14]:
deck = torch.load("results/distilbert-base-uncased-ver8/distilbert-base-uncased-focal-AdamW-lr1e-05-ver8-os10_4-deck1.pth")

In [15]:
ds_lv7 = trainer.dsgen.valid_lv7().dataset

In [16]:
text, otext = ds_lv7[0]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[ -4.1111,   7.0637, -11.9776,  -5.6156, -14.1286,  -5.7573, -13.3618]])


In [17]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[44.9534, 46.0955, 53.5175, 53.5449]]),
 tensor([[254, 442,  43, 629]]),
 tensor([[0, 0, 0, 0]]))

In [18]:
text, otext = ds_lv7[1]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[-3.4819, -2.8398, -9.2523,  1.6889, -9.6386, -1.5814, -9.2860]])


In [19]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[41.6755, 48.6192, 49.1337, 56.9712]]),
 tensor([[248, 442, 626, 558]]),
 tensor([[0, 0, 0, 0]]))

In [20]:
text, otext = ds_lv7[2]
activation = []
print(model(text[None].cuda())[0].cpu())

tensor([[  3.8556,   0.5571, -12.1813,  -4.8405, -14.1694,  -5.0412, -13.1378]])


In [21]:
dists, indices = torch.norm(deck["feat"] - activation[0][None], p=None, dim=1).topk(4, largest=False)
dists, indices, deck["tlevel"][indices]

(tensor([[48.0951, 48.5695, 57.3186, 59.6866]]),
 tensor([[254, 626, 255, 518]]),
 tensor([[0, 0, 0, 0]]))

# 덱만들기 (Submission)

In [22]:
ds_test = trainer.dsgen.test()

In [23]:
activation = []
deck = {"feat": [], "otext": [], "fclevel": [], "ids": []}
with tqdm(total=len(ds_test), ncols=100, file=sys.stdout) as t:
    for i in range(len(ds_test)):
        text, otext, ids = ds_test[i]
        fclevel = model(text[None].cuda(non_blocking=True))[0].argmax(dim=1).cpu()
        deck["fclevel"].append(fclevel)
        deck["otext"].append(otext)
        deck["ids"].append(ids)
        t.update()

 15%|███████▋                                           | 164600/1095951 [19:34<1:50:44, 140.17it/s]


KeyboardInterrupt: 

In [ ]:
deck["feat"] = torch.stack(activation)
deck["fclevel"] = torch.stack(deck["fclevel"])

In [ ]:
deck["feat"] = deck["feat"][:, 0, :]
deck["fclevel"] = deck["fclevel"][:, 0]

In [ ]:
deck["feat"].shape, deck["fclevel"].shape

In [ ]:
torch.save(deck, outdir / f"{postfix}-deck2.pth")

In [13]:
deck1 = torch.load(outdir / f"{postfix}-deck1.pth")

In [33]:
ds_test[0]

(tensor([  101, 11382, 19445,  1024,  1063,  1000,  2828,  1000,  1024,  1000,
          7561,  1000,  1010,  1000, 22073,  1000,  1024,  1031,  1000,  5432,
          1000,  1010,  1000, 26319,  1011,  3074,  1000,  1033,  1010,  1000,
          2504,  1000,  1024,  1000,  7561,  1000,  1010,  1000,  7561,  1000,
          1024,  1063,  1000,  4471,  1000,  1024,  1000,  1031,  3945,  1035,
          4403,  1035,  7781,  1035,  6453,  1033,  2035, 23327,  3478,  1000,
          1010,  1000,  2171,  1000,  1024,  1000,  7561,  1000,  1010,  1000,
          9991,  1000,  1024,  1000,  1031,  3945,  1035,  4403,  1035,  7781,
          1035,  6453,  1033,  2035, 23327,  3478,  1024,  1024,  1063,  1032,
          1000,  4130,  1032,  1000,  1024,  1032,  1000,  1013,  1012, 11382,
         19445,  1035,  4708,  1035,  3208,  1013,  1035,  9986,  1013,  1035,
          3945,  1032,  1000,  1010,  1032,  1000, 23032,  1032,  1000,  1024,
          1063,  1032,  1000,  8568,  1035, 20165,  